In [1]:
import pandas as pd
import numpy as np

In [2]:
all_player_stats = pd.read_csv('player_stats.csv')

To resolve the issue of discrepancies in playing time among players, we will use per games statistics.

In [3]:
all_player_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4504 entries, 0 to 4503
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4504 non-null   int64  
 1   Player      4504 non-null   object 
 2   Pos         4504 non-null   object 
 3   G           4390 non-null   float64
 4   GS          3181 non-null   float64
 5   MP          4233 non-null   float64
 6   FG          4390 non-null   float64
 7   FGA         4390 non-null   float64
 8   FG%         4372 non-null   float64
 9   3P          3373 non-null   float64
 10  3PA         3373 non-null   float64
 11  3P%         2964 non-null   float64
 12  2P          3452 non-null   float64
 13  2PA         3452 non-null   float64
 14  2P%         3423 non-null   float64
 15  eFG%        3435 non-null   float64
 16  FT          4390 non-null   float64
 17  FTA         4390 non-null   float64
 18  FT%         4205 non-null   float64
 19  ORB         3630 non-null  

Keeping things consistent - some players played in the NBA, ABA, BAA so we're only considering those who played in the NBA and their respective NBA stats
Removed rows where all stats are blank (these are players who have never played in the NBA - only played in ABA or BAA)

In [4]:
subset = ['G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB',
          'AST','STL','BLK','TOV','PF','PTS']
all_player_stats.dropna(subset=subset,axis =0,inplace=True, how ='all') 

Some players are assigned multiple positions because they played different positions for different teams. Since we are using the positions as our targets, we will need to narrow it down to one. We will use the first assigned position listed since that is the position that the player held the most throughout their career.

In [5]:
all_player_stats.Pos.unique()

array(['Power Forward', 'Center and Power Forward', 'Center',
       'Point Guard', 'Shooting Guard',
       'Power Forward, Small Forward, and Center',
       'Small Forward and Power Forward', 'Guard', 'Small Forward',
       'Power Forward and Small Forward',
       'Shooting Guard and Small Forward',
       'Shooting Guard and Point Guard', 'Power Forward and Center',
       'Point Guard and Shooting Guard',
       'Shooting Guard, Small Forward, and Point Guard',
       'Small Forward and Shooting Guard', 'Forward',
       'Power Forward, Small Forward, Shooting Guard, and Point Guard',
       'Center/Forward', 'Guard/Forward', 'Forward/Center',
       'Small Forward and Point Guard',
       'Small Forward, Shooting Guard, and Power Forward',
       'Center, Power Forward, and Small Forward', 'Forward/Guard',
       'Small Forward, Power Forward, and Center',
       'Shooting Guard, Small Forward, and Power Forward',
       'Small Forward, Power Forward, and Shooting Guard',
     

Shooting Guard is the position with the most letters - can use this to filter for players that are assigned multiple positions. Only pull the first position

In [6]:
# players categorized with more than one position
all_player_stats[all_player_stats['Pos'].apply(len) > len('Shooting Guard')]

,Unnamed: 0,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Zaid Abdul-Aziz,Center and Power Forward,505.0,NaN,21.8,3.5,8.2,0.428,NaN,...,0.728,2.4,5.4,8.0,1.2,0.6,1.0,0.9,2.2,9.0
5,5,Shareef Abdur-Rahim,"Power Forward, Small Forward, and Center",830.0,704.0,34.8,6.5,13.9,0.472,0.2,...,0.810,2.3,5.3,7.5,2.5,1.0,0.8,2.6,2.8,18.1
6,6,Tom Abernethy,Small Forward and Power Forward,319.0,NaN,17.0,2.3,4.6,0.492,0.0,...,0.747,1.2,2.0,3.2,1.2,0.6,0.2,0.5,1.6,5.6
14,14,Quincy Acy,Power Forward and Small Forward,337.0,60.0,16.0,1.7,3.9,0.444,0.5,...,0.759,0.9,2.6,3.5,0.6,0.4,0.4,0.6,2.0,4.9
15,15,Alvan Adams,Center and Power Forward,988.0,342.0,27.5,5.8,11.6,0.498,0.0,...,0.788,2.0,5.0,7.0,4.1,1.3,0.8,2.6,3.3,14.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4451,4451,Brandan Wright,Power Forward and Center,428.0,62.0,16.2,3.0,4.9,0.607,0.0,...,0.674,1.4,2.3,3.6,0.5,0.5,1.0,0.5,1.3,7.0
4454,4454,Delon Wright,Point Guard and Shooting Guard,319.0,54.0,20.9,2.8,6.2,0.451,0.7,...,0.795,0.8,2.4,3.2,3.1,1.1,0.4,1.0,1.2,7.6
4465,4465,Tony Wroten,Point Guard and Shooting Guard,145.0,34.0,21.2,4.0,9.6,0.413,0.6,...,0.647,0.7,1.8,2.5,3.0,0.9,0.2,2.6,1.8,11.1
4481,4481,Sam Young,Small Forward and Shooting Guard,249.0,52.0,15.9,2.2,5.1,0.442,0.2,...,0.742,0.7,1.6,2.3,0.7,0.5,0.2,0.8,1.2,5.8


In [7]:
# players categorized one position
all_player_stats[all_player_stats['Pos'].apply(len) <= len('Shooting Guard')]

,Unnamed: 0,Player,Pos,G,GS,MP,FG,FGA,FG%,3P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,Alaa Abdelnaby,Power Forward,256.0,53.0,12.5,2.4,4.8,0.502,0.0,...,0.701,1.1,2.2,3.3,0.3,0.3,0.3,1.0,1.9,5.7
2,2,Kareem Abdul-Jabbar,Center,1560.0,625.0,36.8,10.2,18.1,0.559,0.0,...,0.721,2.4,7.6,11.2,3.6,0.9,2.6,2.7,3.0,24.6
3,3,Mahmoud Abdul-Rauf,Point Guard,586.0,336.0,26.7,6.0,13.6,0.442,0.8,...,0.905,0.4,1.5,1.9,3.5,0.8,0.1,1.6,1.9,14.6
4,4,Tariq Abdul-Wahad,Shooting Guard,236.0,145.0,20.4,3.1,7.3,0.417,0.1,...,0.703,1.2,2.1,3.3,1.1,0.8,0.4,1.3,2.1,7.8
7,7,Forest Able,Guard,1.0,NaN,1.0,0.0,2.0,0.000,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4499,4499,Paul Zipser,Small Forward,98.0,30.0,17.0,1.7,4.6,0.371,0.7,...,0.769,0.3,2.3,2.6,0.8,0.4,0.3,0.8,1.7,4.7
4500,4500,Ante Žižić,Center,113.0,27.0,13.4,2.4,4.2,0.581,0.0,...,0.711,1.3,2.6,3.9,0.6,0.2,0.4,0.7,1.5,6.0
4501,4501,Jim Zoet,Center,7.0,0.0,4.3,0.1,0.7,0.200,0.0,...,NaN,0.4,0.7,1.1,0.1,0.1,0.4,0.6,1.3,0.3
4502,4502,Bill Zopf,Point Guard,53.0,NaN,7.5,0.9,2.5,0.363,NaN,...,0.556,NaN,NaN,0.9,1.4,NaN,NaN,NaN,0.6,2.2


Below code takes care of the rows that have multiple positions listed out with commas, have ands or slashes - more cleaning need to be done
Cannot split by '/' and remove with the below code - need both positions to decide what position it is (i.e. if we take the first position it might generically say guard - won't know which guard, second position gives us more insight. Guard/Forward -> Shooting Guard)

In [8]:
# Takes care of the rows that have multiple positions listed out with commas, have ands or slashes - more cleaning need to be done
all_player_stats['Target'] = all_player_stats['Pos'].apply(lambda x: x.split(',')[0] if ',' in x else x.split(',')[0].split(' and ')[0])


In [9]:
all_player_stats['Target'].unique()

array(['Power Forward', 'Center', 'Point Guard', 'Shooting Guard',
       'Small Forward', 'Guard', 'Forward', 'Center/Forward',
       'Guard/Forward', 'Forward/Center', 'Forward/Guard',
       'Point Guard\n'], dtype=object)

There are 156 players that are categorized generically as a guard or a forward - hard to put them in a position without looking at them individually and make judgement calls on my end. Will be dropping these players from the dataset which is only about 3.4%

In [31]:
index = all_player_stats.index
condition = (all_player_stats['Target']=='Guard') | (all_player_stats['Target']=='Forward')
rows_to_drop_indices = index[condition]
all_player_stats.drop(rows_to_drop_indices, axis = 0, inplace=True)

In [32]:
all_player_stats['Target'].unique()

array(['Power Forward', 'Center', 'Point Guard', 'Shooting Guard',
       'Small Forward', 'Center/Forward', 'Guard/Forward',
       'Forward/Center', 'Forward/Guard', 'Point Guard\n'], dtype=object)

Further cleaning the Target column - classifying those with '/' with the first position and changing guard to the corresponding position etc.

In [11]:
guard/forward -> small forward shooting guard
forward/guard -> small forward shooting guard
center/forward -> power forward 

SyntaxError: invalid syntax (<ipython-input-11-362fc17de2c6>, line 1)